In [7]:
import sys
sys.path.append('../')
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np
import torch
import yaml
import cv2

from brutils import Namespace
from torch.utils.data import DataLoader
from traj_dataset import TrajDataset
from models import * 

In [8]:
X_embedded = np.load('viz_cache/X_embedded.npz')
contexts = np.load('viz_cache/contexts.npz')

In [9]:
import plotly.graph_objs as go
fig = go.FigureWidget(go.Scatter3d(
                          x=X_embedded[:,0],
                          y=X_embedded[:,1],
                          z=X_embedded[:,2],
                          mode='markers'
                          )
                      )

In [10]:
#from plotly.callbacks import Points, InputDeviceState
#points, state = Points(), InputDeviceState()
scatter = fig['data'][0]

color_weights = np.arange(0,100,100/12600)
scatter['marker'] = dict({'size':1,'color':color_weights,'colorscale':'Viridis'})
fig.layout.title = 't-SNE dims=3, perplexity=200, num_iter=5000'
fig.layout.hovermode='closest'
scatter.text = [str(i) for i in range(0,12600)]
scatter.hoverinfo = 'text'

from ipywidgets import HTML
details = HTML()
details

from ipywidgets import Image, Layout
success, enc_img = cv2.imencode('.png', np.stack((contexts[0,:,:])*3, axis=-1))
image_widget = Image(
    value = enc_img.tobytes(),
    layout = Layout(height='300px', width='300px')
)
image_widget

def hover_fn(trace, points, state):

    ind = points.point_inds[0]
    
    # Update details HTML widget
    details.value = str(ind)
    success, enc_img = cv2.imencode('.png', np.stack((contexts[ind,:,:],)*3, axis=-1))
    image_widget.value = enc_img.tobytes()

scatter.on_hover(hover_fn)


In [26]:
from ipywidgets import HBox, VBox
HBox([fig,VBox([image_widget],layout=Layout(padding='100px 10px 10px 10px'))])

    'data': [{'hoverinfo': 'text',
              'marker': {'color': array([0.00…

In [ ]:
import plotly.io as pio
pio.write_html(fig, 'index.html')